In [1]:
from pyspark.sql import functions as F
from service.utils.spark import get_spark_session
spark = get_spark_session(dev=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/27 13:15:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
test_namespace = 'gold' 
# spark.sql(f"DROP NAMESPACE IF EXISTS {test_namespace} CASCADE")
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {test_namespace}").show()

++
||
++
++



In [3]:
TARGET_CATEGORY = 'health_beauty'

In [4]:
sale_stats = spark.read.table('warehousedev.gold.sale_stats')
sale_stats.show(n=5)

+--------------------+---------------+--------------------+-----------+-----------+---------+
|          product_id|       category|           seller_id|order_count|total_sales|mean_sale|
+--------------------+---------------+--------------------+-----------+-----------+---------+
|aca2eb7d00ea1a7b8...|furniture_decor|955fee9216a65b617...|        520|   44197.75|  84.9957|
|422879e10f4668299...|   garden_tools|1f50f920176fa81da...|        484|   34201.26|  70.6638|
|99a4788cb24856965...| bed_bath_table|4a3ca9315b744ce9f...|        471|   49338.25| 104.7521|
|389d119b48cf3043d...|   garden_tools|1f50f920176fa81da...|        390|   28543.65|  73.1888|
|368c6c730842d7801...|   garden_tools|1f50f920176fa81da...|        388|    27984.4|  72.1247|
+--------------------+---------------+--------------------+-----------+-----------+---------+
only showing top 5 rows



In [5]:
target_product_sales_stats = sale_stats.filter(F.col('category') == TARGET_CATEGORY)

In [6]:
# 1. 기준점(Threshold) 계산
# percentile_approx 함수를 사용하여 분위수 계산
order_count_threshold = target_product_sales_stats.agg(
    F.expr("percentile_approx(order_count, 0.75)")
).collect()[0][0]

median_avg_price = target_product_sales_stats.agg(
    F.expr("percentile_approx(mean_sale, 0.5)")
).collect()[0][0]

# print(order_count_threshold)
# print(median_avg_price)


# 2. 'group' 컬럼 추가
# when/otherwise와 col 함수를 사용하여 조건에 따라 그룹을 분류합니다.
health_beauty_sales_stats_bcg = target_product_sales_stats.withColumn("segment",
    F.when((F.col("order_count") >= order_count_threshold) & (F.col("mean_sale") >= median_avg_price), "Star Products")
    .when((F.col("order_count") >= order_count_threshold) & (F.col("mean_sale") < median_avg_price), "Volume Drivers")
    .when((F.col("order_count") < order_count_threshold) & (F.col("mean_sale") >= median_avg_price), "Niche Gems")
    .otherwise("Question Marks")
)

health_beauty_sales_stats_bcg.writeTo(f"{test_namespace}.health_beauty_sales_stats_bcg").createOrReplace()

In [7]:
spark.stop()